In [ ]:
import numpy as np
import pandas as pd

from scipy import signal
from scipy import integrate
from scipy import fftpack

import matplotlib.pyplot as plt

import utils
import mac
import oma
import rfp_utils

# plt.style.use('seaborn-pastel')

# 1) Simulación de edificio de excitado en la base

In [ ]:
# Edificio de 4 pisos de 20 toneladas cada uno, con rigideces laterales de 57MN/m entre si
k = 57e6 # N/m
m = 20e3 # kg
k1, k2, k3, k4 = k, k, k, k
m1, m2, m3, m4 = m, m, m, m

# Matrices de rigidez y de masa
K = np.array([[k1+k2, -k2, 0, 0],
             [-k2, k2+k3, -k3, 0],
             [0, -k3, k3+k4, -k4],
             [0, 0, -k4, k4]])

M = np.diag((m1, m2, m3, m4))

print("K:")
utils.printMatrix(K)
print("M:")
utils.printMatrix(M)

In [ ]:
# Resolución por autovectores y autovalores
A = np.linalg.inv(M)@K
lamb, eigv = np.linalg.eig(A)
lamb = np.flip(lamb) # menor a mayor
eigv = np.flip(eigv, axis=1)

In [ ]:
# Frecuencias modales
w_modes = np.sqrt(lamb)
f_modes = w_modes/2/np.pi
mode_num = np.arange(len(w_modes))
print(('f = {}').format(np.round(f_modes,3)))

In [ ]:
# Matriz de formas modales
print("Formas modales:")
utils.printMatrix(eigv)

In [ ]:
# Matriz de masas modales
M_modal = np.transpose(eigv)@M@eigv
M_modal[M_modal<1e-3] = np.nan
print("Masas modales:")
utils.printMatrix(M_modal)

In [ ]:
# Matriz de rigideces modales
K_modal = np.transpose(eigv)@K@eigv
K_modal[K_modal<1e-3] = np.nan
print("Rigideces modales:")
utils.printMatrix(K_modal)

In [ ]:
# Normalización de vectores modales
eigv_norm = np.zeros(eigv.shape)
for mode in range(len(w_modes)):
    eigv_norm[:,mode] = eigv[:,mode]/max(abs(eigv[:,mode]))

In [ ]:
# MAC teórico
MAC_theo = mac.get_MAC_matrix(eigv_norm, eigv_norm)
mac.plot_MAC(MAC_theo, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(mac.get_max_off_diagonal(MAC_theo)))

In [ ]:
utils.plot_modes_complexity(eigv_norm)

In [ ]:
utils.plot_column_modes(eigv_norm)

## 1.1) Simulación de aceleración con perfil de ruido blanco

In [ ]:
# Aceleración maxima en funcion del tiempo
def max_accel(t,t_end):
    a = 0.5*(1-np.cos(2*np.pi*t/t_end))
    return a

In [ ]:
# Simulación de aceleracion en forma de ruido blanco bajo la curva de aceleracion maxima
t_end = 3000
delta_t = 1/119
t = np.linspace(0, t_end, int(t_end/delta_t))

R = 0.1*np.random.normal(size=len(t)) # Vector de ruido gaussiano
R_mean = np.mean(R)

Ug_time = (R - R_mean)*max_accel(t, t_end) # Aceleración escalada con ruido
max_accel_vec = max_accel(t, t_end) # Vector con aceleraciones puras

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
fig.add_axes()
ax.plot(t, Ug_time, color='k', marker='')
ax.plot(t, max_accel_vec, color='r')
plt.title('Historia de aceleraciones en la base')
plt.xlabel('Tiempo [s]')
plt.ylabel('Aceleración [g]')
ax.grid(True, markevery=1)

In [ ]:
# PSD de aceleraciones en la base
nperseg = 1024
freq, Ug_psd = signal.welch(Ug_time, 
                      fs=1./(delta_t), # sample rate
                      window='hamming', # apply a Hanning window before taking the DFT
                      nperseg=nperseg, # compute periodograms of 256-long segments of x
                      noverlap=nperseg//2,
                      detrend='constant',
                      return_onesided=False) #'constant') # detrend x by subtracting the mean)

plt.figure()
plt.semilogy(freq[1:len(Ug_psd)//2], Ug_psd[1:len(Ug_psd)//2])
plt.title('PSD de aceleración en la base')
plt.ylim([np.min(Ug_psd[1:]), np.max(Ug_psd[1:])])
plt.xlabel('Frequencia [Hz]')
plt.ylabel('ASD [g²/Hz]')
plt.grid()
plt.show()

In [ ]:
# Transformada de Fourier de aceleraciones en la base
Ug_fft = np.fft.fft(Ug_time) * delta_t
Ug_fft_freq = np.fft.fftfreq(t.shape[-1], d=delta_t)

plt.figure()
plt.semilogy(Ug_fft_freq[:len(Ug_fft_freq)//2], abs(Ug_fft[:len(Ug_fft_freq)//2]))
plt.title('Espectro de aceleración en la base')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Espectro de aceleraciones [g/Hz]')
plt.show()

## 1.2) Simulación de respuesta dinámica del edificio a ruido blanco su base

In [ ]:
xi = 0.025 # fracción de amortiguamiento crítico
Xi = xi*np.ones(eigv_norm.shape[1])
r = np.array([1, 1, 1, 1]).reshape(4,1) # vector logico de desplazamientos respecto de la base
print('xi = {}'.format(Xi[:]))
print('r = {}^T'.format(r[:,0]))

La solucion del espectro de desplazamientos para cada modo es
<br/>
<center> $Y(\omega) = \frac{\frac{\iota}{m_i} U_g(\omega)}{\omega_i^2 - \omega^2 + 2i \xi_i \omega_i \omega}$ </center>
<br/>
con
<center> $\iota = \Phi^T M r$. </center>
<br/>
Y la aceleración es
<center> $\ddot{Y}(\omega) = \omega^2 Y(\omega)$ </center>

In [ ]:
U_g = np.copy(Ug_fft)
w = 2*np.pi*Ug_fft_freq
f = w/2/np.pi

In [ ]:
total_dofs = eigv.shape[0]
total_modes = eigv.shape[1]

In [ ]:
I = np.transpose(eigv)@M@r
m_modal = M_modal.diagonal() 
Y = np.zeros((len(w), total_modes), dtype=complex)
ddotY = np.copy(Y) 
for mode in range(total_modes):
    C = I[mode] / (m_modal[mode]*(w_modes[mode]**2 - w**2 + 2*1j*Xi[mode]*w_modes[mode]*w))
    Y[:,mode] = C*Ug_fft
    ddotY[:,mode] = -(w**2) * Y[:, mode]

In [ ]:
plt.figure()
lgnd  = ['modo {}'.format(mode+1) for mode in range(total_modes)]
plt.semilogy(f[:len(f)//4], abs(ddotY[:len(f)//4]))
plt.title('Aceleraciones modales')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('Espectro de Aceleraciones [g/Hz]')
plt.ylim([abs(ddotY[len(f)//4, 0]), None])
plt.legend(lgnd, loc='upper right')
plt.show()
print(np.max(abs(ddotY)))

In [ ]:
freq_max = [abs(f[i]) for i in ddotY.argmax(axis=0)]
print(('freqs from max response = {}').format(np.round(freq_max, 3)))
print('errors in % = {}'.format(np.round(100*(f_modes - freq_max) / f_modes, 2)))

### 1.2.1) Respuesta en cada piso

In [ ]:
# Espectro de respuesta en frecuencia
X = np.copy(Y)
X = (eigv @ Y.T).T # Desplazamientos en los grados de libertad
ddotX = -(w**2 * X.T).T # Aceleraciones

In [ ]:
plt.figure()
lgnd  = ['piso {}'.format(mode+1) for mode in range(total_modes)]
plt.semilogy(f[:len(f)//4], abs(ddotX[:len(f)//4,:]))
plt.title('Aceleraciones por piso')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('Espectro de aceleraciones [g/Hz]')
plt.ylim([abs(ddotX[len(f)//4, 0]), None])
plt.legend(lgnd, loc='upper right')
plt.show()

# 2. Operational Modal Analysis

## 2.1) Peak-picking (PP)

In [ ]:
ddotX_psd = np.copy(ddotX)
for piso in range(total_dofs):
    ddotX_psd[:, piso] = ddotX[:, piso] * np.conj(ddotX[:, piso])

In [ ]:
# Average Normalized PSD (ANPSD)
ANPSD = np.zeros(ddotX_psd.shape[0], dtype=complex)
NPSD = np.copy(ddotX_psd)
for piso in range(total_dofs):
    NPSD[:, piso] = ddotX_psd[:, piso] / np.sum(ddotX_psd[:, piso])
ANPSD = np.sum(NPSD, axis=1)
mode_ind_pp = np.array([m for m in signal.argrelmax(ANPSD[:len(f)//4], order=4000)]).flatten()

In [ ]:
plt.figure()
plt.semilogy(f[:len(f)//4], abs(ANPSD[:len(f)//4]), label='ANPSD')
plt.scatter(f[mode_ind_pp], abs(ANPSD[mode_ind_pp]), label='modes', color='r')
plt.title('Average Normalized PSD')
# plt.ylim([abs(ANPSD[len(f)//4]), None])
plt.xlabel('Frequencia [Hz]')
plt.ylabel('ASD [g²/Hz]')
plt.legend()
plt.show()

In [ ]:
f_pp = f[mode_ind_pp[:4]]
err_f_pp = (f_pp - f_modes)/f_modes*100

data = np.vstack((mode_num, f_modes, f_pp)).T
headers = ['Modo', 'Teoría [Hz]', 'Peak-picking [Hz]']
utils.print_modes_dataframe(data=data, headers=headers, decimals=2)

### 2.1.1) Formas modales

In [ ]:
angle_th = 30
mode_idxes_pp = mode_ind_pp[:total_modes]
modes_pp = oma.get_peak_picking_modes(psd=ddotX, angle_th=angle_th, mode_idxes=mode_idxes_pp)

In [ ]:
MAC_pp = mac.get_MAC_matrix(eigv_norm, modes_pp)
mac.plot_MAC(MAC_pp, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(mac.get_max_off_diagonal(MAC_pp)))

In [ ]:
utils.plot_modes_complexity(modes_pp)

## 2.2) Frequency Domain Decomposition (FDD)

In [ ]:
# Respuesta en el tiempo
ddotX_time = fftpack.ifft(ddotX, axis=0) # Obtengo respuestas en el tiempo mediante transformada inversa

# Matriz de densidades espectrales cruzadas de la respuesta.
csd_nperseg = 1024
S_xx = np.zeros((csd_nperseg, total_dofs, total_dofs), dtype=complex)
for dof1 in range(total_dofs):
    for dof2 in range(total_dofs):
        f_S, S_xx[:, dof1, dof2] = signal.csd(ddotX_time[:, dof1], ddotX_time[:, dof2],
                                               fs=1./(delta_t),
                                               window='hanning',
                                               nperseg=csd_nperseg,
                                               detrend='constant',
                                               axis=0,
                                               return_onesided=False)

In [ ]:
# Extraigo frecuencias y formas de modo por SVD
u_fdd, s_fdd, vh_fdd = np.linalg.svd(S_xx)
    
# Maximos del primer valor singular
mode_ind_fdd = np.array([m for m in signal.argrelmax(s_fdd[:len(f_S)//4, 0], order=12)]).flatten()

In [ ]:
plt.semilogy(f_S[:len(f_S)//4], s_fdd[:len(f_S)//4, :])
# plt.ylim([s[len(f_S)//4, 0], None])
plt.scatter(f_S[mode_ind_fdd],
            abs(s_fdd[mode_ind_fdd, 0]),
            color='r')
plt.title('Valores singulares de la matriz de PSD')
plt.ylabel('ASD [g^2/Hz]')
plt.xlabel('Frequencia [Hz]')
plt.show()

In [ ]:
f_fdd = f_S[mode_ind_fdd[:4]]
err_f_fdd = (f_fdd - f_modes)/f_modes*100

data = np.vstack((mode_num, f_modes, f_fdd)).T
headers = ['Modo', 'Teoría [Hz]', 'FDD [Hz]']
utils.print_modes_dataframe(data=data, headers=headers, decimals=2)

In [ ]:
# Formas modales
modes_fdd = np.zeros((total_dofs, total_modes), dtype=complex)
for idx in range(modes_fdd.shape[1]):
    modes_fdd[:, idx] = u_fdd[mode_ind_fdd[idx], :, 0]

# Normalización
for col in range(modes_fdd.shape[1]):
    modes_fdd[:,col] = modes_fdd[:,col]/max(abs(modes_fdd[:,col]))

In [ ]:
MAC_fdd = mac.get_MAC_matrix(eigv_norm, modes_fdd)
mac.plot_MAC(MAC_fdd, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(mac.get_max_off_diagonal(MAC_fdd)))

In [ ]:
utils.plot_modes_complexity(modes_fdd)

## 2.3) Enhanced Frequency Domain Decomposition (EFDD)

### 2.3.1) Identificación de las PSD modales 

In [ ]:
freq = f_S
mac_th = 0.9 #
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:total_modes]
sdof_psd = np.zeros((freq.size, total_modes), dtype=complex)
sdof_acf = np.zeros((freq.size, total_modes), dtype=complex)
t_sdof_acf = np.linspace(0, sdof_psd.shape[0]*delta_t, sdof_psd.shape[0])

fig_psd, ax_psd = plt.subplots()
ax_psd.set_title('Picos de PSD seleccionados por EFDD')
ax_psd.set_xlabel('Frequencia [Hz]')
ax_psd.set_ylabel('PSD [g^2/Hz]')
ax_psd.grid(True, markevery=1)

fig_decay, ax_decay = plt.subplots(total_modes//2, 2)
fig_decay.suptitle('Decaimientos de los sistemas 1-DOF asociados a cada pico')
[ax_decay.flat[i].grid(True, markevery=1) for i in range(len(ax_decay.flat))]

for mode in range(total_modes):
    peak_idx = mode_ind_fdd[mode]
    idx_low, idx_high = oma.get_efdd_segment(u_fdd, peak_idx, mac_th)
  
    ax_psd.semilogy(f_S[idx_low:idx_high],
                    abs(s_fdd[idx_low:idx_high, 0]),
                    color=colors[mode])
    
    sdof_psd[idx_low:idx_high, mode] = s_fdd[idx_low:idx_high, 0]
    sdof_acf[:, mode] = fftpack.ifft(sdof_psd[:, mode])
    ax_decay.flat[mode].plot(t_sdof_acf[:len(t_sdof_acf)//2],
                             sdof_acf[:len(t_sdof_acf)//2, mode].real,
                             color=colors[mode])
    ax_decay.flat[mode].set_xlabel('Tiempo [s]')
plt.show()

### 2.3.2) Estimación de los amortiguamientos

In [ ]:
xi_efdd = np.zeros(total_modes)
R2 = np.zeros(total_modes)

for mode in range(total_modes):
    decay = sdof_acf[:, mode].real[:len(t_sdof_acf)//2]
    xi_efdd[mode], R2[mode] = oma.get_damp_from_decay(decay)

In [ ]:
data = np.vstack((mode_num, Xi, xi_efdd)).T
headers = ['Modo', 'Teoría', 'EFDD']
utils.print_modes_dataframe(data=data, headers=headers, decimals=3)

### 2.3.3) Estimación de las frecuencias

In [ ]:
f_efdd = np.zeros(total_modes)

fig_decay, ax_decay = plt.subplots(total_modes//2, 2)
fig_decay.suptitle('Ajuste de los decaimientos como logaritmicos')

[ax_decay.flat[i].grid(True, markevery=1) for i in range(len(ax_decay.flat))]

for mode in range(total_modes):
    decay = sdof_acf[:, mode].real[:len(t_sdof_acf)//2]
    f_efdd_damped = oma.get_freq_from_signal(t_sdof_acf, sdof_acf[:, mode])
    f_efdd[mode] = f_efdd_damped / np.sqrt(1-xi_efdd[mode]**2)
    w_n_efdd = 2*np.pi*f_efdd[mode]
    peak_ind = np.array([m for m in signal.argrelmax(abs(decay), order=1)]).flatten()

    ax_decay.flat[mode].plot(t_sdof_acf[:len(sdof_acf)//2], decay, color=colors[mode])
    ax_decay.flat[mode].plot(t_sdof_acf[peak_ind],
                             decay[0]*np.exp(-xi_efdd[mode]*w_n_efdd*t_sdof_acf[peak_ind]),
                             color=colors[mode])
    ax_decay.flat[mode].set_xlabel('Tiempo [s]')

In [ ]:
err_f_efdd = (f_efdd - f_modes)/f_modes*100

data = np.vstack((mode_num, f_modes, f_fdd, f_efdd)).T
headers = ['Modo', 'Teoría [Hz]', 'FDD [Hz]', 'EFDD [Hz]']
utils.print_modes_dataframe(data=data, headers=headers, decimals=2)

### 2.3.4) Estimación de formas modales promedio

In [ ]:
# Promedio pesado segun MAC de las formas modales dentro del "segmento EFDD" de cada pico
modes_efdd = np.zeros((total_dofs,total_modes), dtype=complex)
for mode in range(total_modes):
    peak_idx = mode_ind_fdd[mode]
    seg_idxes = oma.get_efdd_segment(u_fdd, peak_idx, mac_th)
    modes_efdd[:, mode] = oma.get_mean_modeshape_efdd(segment_idxes=seg_idxes,
                                                      peak_idx=peak_idx,
                                                      s_vectors=u_fdd)

# Normalización
for col in range(modes_efdd.shape[1]):
    modes_efdd[:,col] = modes_efdd[:,col]/max(abs(modes_efdd[:,col]))

In [ ]:
MAC_efdd = mac.get_MAC_matrix(eigv_norm, modes_efdd)
mac.plot_MAC(MAC_efdd, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(mac.get_max_off_diagonal(MAC_efdd)))

In [ ]:
utils.plot_modes_complexity(modes_efdd)

## 2.4) Curve-fit Frequency Domain Decomposition (CFDD)

### 2.4.1) Estimación mediante cuadrado de la FRF

In [ ]:
mac_th = 0.9
freq = f_S
f_cfdd = np.zeros(total_modes)
xi_cfdd = np.zeros(total_modes)

freq_hat = np.linspace(0, f[len(f)//2-1], 4000)
psd_hat = np.zeros((freq_hat.size, total_modes))

for mode in range(total_modes):
    peak_idx = mode_ind_fdd[mode]
    efdd_idxes = oma.get_efdd_segment(u_fdd, peak_idx, mac_th)
    f_cfdd[mode], xi_cfdd[mode], psd_hat[:, mode] = oma.curve_fit_psd_peak(freq, s_fdd[:, 0], efdd_idxes, freq_hat)
    
    plt.semilogy(freq[efdd_idxes[0]:efdd_idxes[1]],
                 abs(s_fdd[efdd_idxes[0]:efdd_idxes[1], 0]),
                 color=colors[mode])
    plt.semilogy(freq_hat[:freq_hat.size//2],
                 abs(psd_hat[:freq_hat.size//2, mode]),
                 color=colors[mode],
                 linestyle='--')

plt.title('Ajuste por CFDD de picos de PSD')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('PSD [g^2/Hz]')
plt.ylim([abs(psd_hat[len(freq_hat)//4, 1]), None])

In [ ]:
err_f_cfdd = (f_cfdd - f_modes)/f_modes*100

data = np.vstack((mode_num, f_modes, f_fdd, f_efdd, f_cfdd)).T
headers = ['Modo', 'Teoría [Hz]', 'FDD [Hz]', 'EFDD [Hz]', 'CFDD [Hz]']
utils.print_modes_dataframe(data=data, headers=headers, decimals=2)

In [ ]:
data = np.vstack((mode_num, Xi, xi_efdd, xi_cfdd)).T
headers = ['Modo', 'Teoría', 'EFDD', 'CFDD']
utils.print_modes_dataframe(data=data, headers=headers, decimals=3)

# 3) Experimental Modal Analysis (EMA)

## 3.1) Estimadores de FRF: H1 y H2

<br/>
<center> $H(\omega) = \frac{X(\omega)}{F(\omega)}$ </center>
<br/>
<center> $H_1(\omega) = \frac{S_{XF}(\omega)}{S_{FF}(\omega)}$ </center>
<br/>
<center> $H_2(\omega) = \frac{S_{XX}(\omega)}{S_{XF}(\omega)}$ </center>
<br/>

In [ ]:
# Densidades espectrales cruzadas entre respuestas e input.
csd_nperseg = 1024
S_xf = np.zeros((csd_nperseg, total_dofs), dtype=complex)
for dof in range(total_dofs):
        f_S, S_xf[:, dof] = signal.csd(ddotX_time[:, dof], Ug_time,
                                               fs=1./(delta_t),
                                               window='hanning',
                                               nperseg=csd_nperseg,
                                               detrend='constant',
                                               axis=0,
                                               return_onesided=False)

plt.semilogy(f_S[:len(f_S)//4], np.abs(S_xf[:len(f_S)//4]))
plt.title('CSDs entre respuestas y excitación')
plt.ylabel('Acelerancia [g/g]')
plt.xlabel('Frequency [Hz]')
plt.show()

In [ ]:
H1 = np.zeros((S_xf.shape[0], total_dofs), dtype=complex)
for dof in range(total_dofs):
    H1[:, dof] = S_xf[:, dof] / Ug_psd

max_ind_H1 = np.array([m for m in signal.argrelmax(np.abs(H1[:len(f_S)//4, 0]), order=10)]).flatten()
print('Frecuencias de pico para H1 = {} Hz'.format(np.round(f_S[max_ind_H1[:4]], 2)))

H2 = np.zeros((S_xf.shape[0], total_dofs), dtype=complex)
for dof in range(total_dofs):
    H2[:, dof] = S_xx[:, dof, dof] / S_xf[:, dof]
max_ind_H2 = np.array([m for m in signal.argrelmax(np.abs(H2[:len(f_S)//4, 3]), order=10)]).flatten()
print('Frecuencias de pico para H2 = {} Hz'.format(np.round(f_S[max_ind_H2[:4]], 2)))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
ax.flat[0].semilogy(f_S[:len(f_S)//4], np.abs(H1[:len(f_S)//4, :]))
ax.flat[0].scatter(f_S[max_ind_H1], np.abs(np.abs(H1[max_ind_H1,3])), color='k')
ax.flat[1].semilogy(f_S[:len(f_S)//4], np.abs(H2[:len(f_S)//4,:]))
ax.flat[1].scatter(f_S[max_ind_H2], np.abs(H2[max_ind_H2,3]), color='k')
for k in range(len(ax.flat)):
    ax.flat[k].set_title('Acelerancias estimadas por H{}'.format(k+1))
    ax.flat[k].set_ylabel('Acelerancia [g/g]')
    ax.flat[k].set_xlabel('Frequency [Hz]')

fig.tight_layout()
plt.show()

## 3.2) Rational Fraction Polynomial (RFP)

In [ ]:
# Conversión de acelerancia a receptancia(FRF) diviendo por el cuadrado de las frecuencias angulares
dof = 2
w_S = 2*np.pi*f_S[1:len(f_S)//2]
H_frf = H2[1:len(f_S)//2, dof]/w_S**2

In [ ]:
plt.semilogy(f_S[:len(f_S)//4], np.abs(H_frf[:len(f_S)//4]))
plt.title('Receptancia del DOF {} estimada por H2'.format(dof))
plt.ylabel('Acelerancia [g/g]')
plt.xlabel('Frequency [Hz]')
plt.show()

In [ ]:
mode_ind_H_frf = np.array([m for m in signal.argrelmax(np.abs(H_frf), order=10)]).flatten()
print('Frecuencias de picos = {} Hz'.format(np.round(f_S[mode_ind_H_frf], 2)))

In [ ]:
idx = np.linspace(90, 160, 61, dtype=int)
omega = w_S[idx]
f_omega = omega/2/np.pi
frf = H_frf[idx]

modal_params, alpha = rfp_utils.rfp(frf, omega, 3)

In [ ]:
plt.semilogy(f_S[1:len(f_S)//4], np.abs(H_frf[1:len(f_S)//4]), color='tab:blue')
plt.semilogy(f_omega, np.abs(alpha), color='tab:blue', marker='o')
plt.title('FRF estimada por RFP entre {} y {} Hz'.format(
            np.round(f_omega[0],1),
            np.round(f_omega[-1]),1))
plt.ylabel('Acelerancia [g/g]')
plt.xlabel('Frequency [Hz]')
plt.show()

In [ ]:
freq_n = modal_params[0]
xi_n = abs(modal_params[1])
print('Freqs = {} Hz'.format(np.round(freq_n, 2)))
print('Damps = {}'.format(np.round(xi_n, 3)))

## 3.3) Global Rational Fraction Polynomial (GRFP)

In [ ]:
idx = np.linspace(20, 80, 61, dtype=int)
omega = w_S[idx]
f_omega = omega/2/np.pi
frf = np.zeros_like(H2[idx, :], dtype=complex)
for dof in range(total_dofs):
    frf[:, dof] = H2[idx, dof]/omega**2

In [ ]:
plt.title('FRF estimadas por H2 entre {} y {} Hz'.format(
            np.round(f_omega[0],1),
            np.round(f_omega[-1]),1))
plt.xlabel('Frequencia [Hz]')
plt.ylabel('FRF [m/g]')
plt.semilogy(f_omega, np.abs(frf))
plt.show()

In [ ]:
# Obtencion del polinomio denominador
denominator, denominator_coeff = rfp_utils.grfp_denominator(frf, omega, 3)

plt.title('Polinomio denominador comun de las FRF')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('FRF [m/g]')
plt.semilogy(f_omega, np.abs(frf))
plt.semilogy(f_omega, np.abs(1/denominator), color='k', linestyle='--', label='polinomio denominador')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Obtencion de los parametros modales de cada FRF
frf_estimates, modal_params = rfp_utils.grfp_parameters(frf, omega, denominator, denominator_coeff, 3)
f_grfp, xi_grfp, modal_mag_n, modal_ang_n = modal_params
modes_grfp = (modal_mag_n*np.exp(1j*modal_ang_n)).T

print('Frequencias: {} Hz'.format(np.round(f_grfp, 2)))
print('Amortiguamientos: {}'.format(np.round(xi_grfp, 3)))
utils.plot_column_modes(modes_grfp)
plt.show()

In [ ]:
top = len(f_grfp)
data = np.vstack((mode_num[:top], f_modes[:top], f_cfdd[:top], f_grfp)).T
headers = ['Modo', 'Teoría [Hz]', 'CFDD [Hz]', 'GRFP [Hz]']
utils.print_modes_dataframe(data=data, headers=headers, decimals=2)

In [ ]:
MAC_grfp = mac.get_MAC_matrix(eigv_norm[:,:top], modes_grfp)
mac.plot_MAC(MAC_grfp, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(mac.get_max_off_diagonal(MAC_grfp)))

In [ ]:
# Grafico de estimaciones de FRF
plt.title('Ajuste por GRFP de las FRF')
plt.xlabel('Frequencia [Hz]')
plt.ylabel('FRF [m/g]')
for dof in range(frf.shape[1]):
    plt.semilogy(f_omega, np.abs(frf[:, dof]), color=colors[dof])
    plt.semilogy(f_omega, np.abs(frf_estimates[:, dof]), color=colors[dof], linestyle='--')
plt.show()